In [2]:
import re
import pandas as pd 
import numpy as np

In [183]:
Posts = pd.read_csv('../../160-Stackoverflow-Data/300000_rows/PostsClean.csv', \
                    dtype = {'LastEditorDisplayName': str, 'Tags': str})

In [153]:
Posts.head()

,Id,PostTypeId,ParentID,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditorDisplayName,LastEditDate,LastActivityDate,CommunityOwnedDate,ClosedDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount
0,48870881,1,NaN,48871194.0,2018-02-19T17:05:48.193,1,50.0,<p>I have a json file with data as below:</p>\...,2804169.0,2804169.0,NaN,2018-02-19T18:42:53.487,2018-02-19T18:53:19.357,NaN,NaN,PHP search array return all values,<php><arrays>,2.0,0,NaN
1,48870882,1,NaN,48870993.0,2018-02-19T17:05:49.097,0,27.0,<p>I am currently working on a web application...,5128923.0,2598770.0,NaN,2018-02-19T17:13:33.920,2018-02-19T17:13:49.300,NaN,NaN,Creating a reactive user input tab,<c#><html><asp.net><.net><visual-studio>,2.0,0,NaN
2,48870883,2,48869547.0,NaN,2018-02-19T17:05:50.073,1,NaN,<p>Try to add hint to the query:</p>\r\n\r\n<p...,8214734.0,8214734.0,NaN,2018-02-20T10:20:21.007,2018-02-20T10:20:21.007,NaN,NaN,NaN,NaN,NaN,2,NaN
3,48870884,2,3865445.0,NaN,2018-02-19T17:05:50.273,0,NaN,In support of Sean's post:\r\n\r\n<p>The place...,2055283.0,NaN,NaN,NaN,2018-02-19T17:05:50.273,NaN,NaN,NaN,NaN,NaN,0,NaN
4,48870885,2,48870539.0,NaN,2018-02-19T17:05:51.300,0,NaN,<p>I discovered the convention (couldn't find ...,1052870.0,NaN,NaN,NaN,2018-02-19T17:05:51.300,NaN,NaN,NaN,NaN,NaN,1,NaN
5,48870886,1,NaN,NaN,2018-02-19T17:05:52.677,0,27.0,<p>So I am trying to create records in mongodb...,8353588.0,NaN,NaN,NaN,2018-02-19T17:18:09.357,NaN,NaN,Mongoose Validation Error,<node.js><mongodb><express><mongoose><mongoose...,1.0,2,NaN
6,48870888,1,NaN,48871145.0,2018-02-19T17:06:01.170,1,17.0,<p>I am trying to find out a way how add a col...,9381432.0,NaN,NaN,NaN,2018-02-19T17:21:55.740,NaN,NaN,R - Calculating annual incrementals across dif...,<r>,1.0,0,NaN
7,48870889,2,48870709.0,NaN,2018-02-19T17:06:01.910,0,NaN,"<p>Group by the user, then aggregate a list of...",3182843.0,NaN,NaN,NaN,2018-02-19T17:06:01.910,NaN,NaN,NaN,NaN,NaN,0,NaN
8,48870890,1,NaN,NaN,2018-02-19T17:06:06.810,0,19.0,<p>I need a simple and clear example on how to...,3613597.0,3613597.0,NaN,2018-02-19T17:18:00.510,2018-02-19T17:18:00.510,NaN,2018-02-19T17:09:58.357,Passing props from parent component to a neste...,<javascript><reactjs><react-native>,0.0,0,NaN
9,48870891,2,48870253.0,NaN,2018-02-19T17:06:16.113,12,NaN,<p>With combination of <code>re.findall()</cod...,3185459.0,3185459.0,NaN,2018-02-19T17:13:55.730,2018-02-19T17:13:55.730,NaN,NaN,NaN,NaN,NaN,8,NaN


In [39]:
Posts[Posts.Id == 48882980]

,Id,PostTypeId,ParentID,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditorDisplayName,LastEditDate,LastActivityDate,CommunityOwnedDate,ClosedDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount
10389,48882980,2,48882635.0,NaN,2018-02-20T10:23:28.970,1,NaN,"<p><a href=""http://php.net/oci_fetch_array"" re...",476.0,NaN,NaN,NaN,2018-02-20T10:23:28.970,NaN,NaN,NaN,NaN,NaN,3,NaN


In [40]:
Posts[Posts.Id == 48882635]

,Id,PostTypeId,ParentID,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditorDisplayName,LastEditDate,LastActivityDate,CommunityOwnedDate,ClosedDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount
10100,48882635,1,NaN,48882980.0,2018-02-20T10:08:24.050,0,26.0,<p>I have a simple PHP coding which fetches da...,6002555.0,230471.0,NaN,2018-02-20T10:17:34.733,2018-02-20T10:23:28.970,NaN,NaN,Why does my PHP code recognises NULL entities ...,<php><oracle>,1.0,1,NaN


## Frequency table of tags

In [184]:
# make a function that extracts tags from Posts' tags, using Posts' Id
def freq_table(idx):
    '''
    @param idx: list
    return series
    '''
    filtered = []
    pattern = '[<>]'
    regex = re.compile(pattern, flags = re.IGNORECASE)
    
    for j in range(len(idx)):
        tags = Posts.loc[(Posts.Id == idx.iloc[j])]['Tags'] #extracting tags from Posts' Id
        
        try:
            tags1 = list(filter(None,regex.split(tags.iloc[0]))) 
            filtered.extend(tags1)
        except:
            continue
    freq_tags = pd.DataFrame({'Tag':filtered}).groupby('Tag').size()
    
    return freq_tags

### Tags of a user's answers

In [185]:
#tags of 'question' posts that the user posted 'answers' to, and count the number of the user's answers for each tag
def answers_user(user_id):
    """
    @param user_id: int 
    return: series 
    """
    idx = Posts.loc[(Posts.OwnerUserId == user_id) &  (Posts.PostTypeId == 2)]['ParentID']
    return freq_table(idx)

In [186]:
#example
answers_user(476)

Tag
angular rest api backend ms-access    1
interface abstraction                 1
javascript                            1
javascript undefined typeof           1
php                                   8
php ftp sftp                          1
php oracle                            1
php postgresql encoding               1
php security                          1
php unicode                           1
python int multiplication             1
python syntax mixins                  1
dtype: int64

### Acceptance ratio of a user's answers

In [187]:
#tags of 'question' posts that the user's answer was accepted to('AcceptedAnswer'), and count the number of the user's accepted answers for each tag
def accepted_ratio(user_id):
    """
    @param user_id: int 
    return: series 
    """
    accepted_tags = []
    pattern = '[<>]'

    regex = re.compile(pattern, flags = re.IGNORECASE)
    answers = Posts.loc[(Posts.OwnerUserId == user_id) &  (Posts.PostTypeId == 2)]

    for i in range(len(answers)):
        tags = Posts.loc[(Posts.AcceptedAnswerId == answers.Id.iloc[i])]['Tags']
        try:
            tags1 = list(filter(None,regex.split(tags.iloc[0]))) 
            accepted_tags.extend(tags1)
        except:
            continue
    freq_acanswers = pd.DataFrame({'Tag':accepted_tags}).groupby('Tag').size()
    return round(freq_acanswers/answers_user(user_id),2).dropna()

In [188]:
#example
accepted_ratio(1850609)

Tag
angular                                         0.67
angular asynchronous                            1.00
angular asynchronous undefined                  1.00
angular typescript angular5                     1.00
button onclick                                  1.00
css vuejs2 components                           1.00
datepicker vuejs2                               1.00
docker                                          1.00
function methods onclick                        1.00
html vuejs2                                     1.00
java vuejs2                                     1.00
javascript                                      0.69
javascript android vuejs2 addeventlistener      1.00
javascript angular typescript observable        1.00
javascript arrays matrix                        1.00
javascript components                           1.00
javascript eslint                               1.00
javascript html css                             1.00
javascript html wordpress                 

### Tags of a user's questions

In [189]:
def question_user(user_id):
    """
    @param user_id: int 
    return: series 
    """
    idx = Posts.loc[(Posts.OwnerUserId == user_id) &  (Posts.PostTypeId == 1)]['Id']
    return freq_table(idx)

In [190]:
question_user(9151888)

Tag
javascript html iframe    1
php                       1
dtype: int64

### Tags of a user's comments

In [191]:
Comments = pd.read_csv('../../160-Stackoverflow-Data/300000_rows/Comments.csv')

In [192]:
Comments.head()

,Id,PostId,Score,Text,CreationDate,UserId
0,85065265,49030489,4,Possible duplicate of [Node.js - require empty...,2018-02-28T13:16:41.767,5734311.0
1,85065268,49030285,0,The problem with this is the bad structure log...,2018-02-28T13:16:45.110,2718972.0
2,85065269,49030414,0,Can you elaborate a little more on ViewModel? ...,2018-02-28T13:16:50.880,9305895.0
3,85065270,49022014,1,In `rollapply` the `width=` can be a one compo...,2018-02-28T13:16:51.247,516548.0
4,85065271,49029381,0,"Yeah, I'm testing here and it doesnt seem to b...",2018-02-28T13:16:52.050,1850609.0


In [193]:
#tags of 'question' posts that the user posted 'comments' to, and count the number of the user's comments for each tag
def comments_user(user_id):
    """
    @param user_id: int 
    return: series 
    """
    idx = Comments.loc[(Comments.UserId == user_id)]['PostId']
    return freq_table(idx)

In [194]:
#example
comments_user(516548)

Tag
python r                                   2
r                                          6
r date xts                                 1
r finance quantmod performanceanalytics    1
r knitr                                    1
r optimization                             1
r plot ggplot2                             1
r sumifs                                   1
dtype: int64

## Total number of activities against tags

In [195]:
# activities include answers, accepted answers and comments
def activities(user_id):
    """
    @param user_id: int 
    return: dataframe
    """
    answers = answers_user(user_id)
    acc_ratio = accepted_ratio(user_id)
    comments = comments_user(user_id)
    questions = question_user(user_id)
    user_table = pd.DataFrame({'Answers' : answers ,'Acceptance Ratio' : acc_ratio ,'Questions':questions,'Comments': comments})
    user_table = user_table.fillna(0)
    if len(user_table) != 0 :
        user_table['Strength'] = user_table.apply(lambda 
                                                  row: (row['Answers']+1/2*row['Comments']+1/2*(row['Questions'])
                                                       )*(row['Acceptance Ratio']), axis=1)
        # As comments are less important than answers or accepted answers, adjusting the impact of comments
    user_table = user_table[user_table.Strength > user_table.Strength.quantile(0.25)] # drop lower 25% strength values 
    user_table.Strength = user_table.Strength / sum(user_table.Strength) #scaling
    return user_table

In [196]:
activities(1850609)

,Acceptance Ratio,Answers,Comments,Questions,Strength
Tag,,,,,
angular,0.67,3.0,2.0,0.0,0.025244
angular asynchronous,1.00,1.0,0.0,0.0,0.009419
angular asynchronous undefined,1.00,1.0,0.0,0.0,0.009419
angular typescript angular5,1.00,1.0,0.0,0.0,0.009419
button onclick,1.00,1.0,0.0,0.0,0.009419
css vuejs2 components,1.00,1.0,0.0,0.0,0.009419
datepicker vuejs2,1.00,1.0,4.0,0.0,0.028258
docker,1.00,1.0,0.0,0.0,0.009419
function methods onclick,1.00,1.0,0.0,0.0,0.009419


In [197]:
activities(9151888)

,Acceptance Ratio,Answers,Comments,Questions,Strength
Tag,,,,,


In [198]:
activities(2336654)

,Acceptance Ratio,Answers,Comments,Questions,Strength
arrays pandas timestamp comparison,1.00,1.0,1.0,0.0,0.037106
pandas dataframe,1.00,1.0,0.0,0.0,0.024737
pandas numpy,1.00,1.0,0.0,0.0,0.024737
python json pandas dataframe,1.00,1.0,0.0,0.0,0.024737
python pandas,0.55,31.0,7.0,0.0,0.469388
python pandas dataframe,0.40,10.0,1.0,0.0,0.103896
python pandas dataframe concatenation,1.00,1.0,1.0,0.0,0.037106
python pandas dataframe mean series,1.00,1.0,0.0,0.0,0.024737
python pandas dataframe merge,0.50,2.0,1.0,0.0,0.030921
python pandas datetime indexing,1.00,1.0,0.0,0.0,0.024737
